<a href="https://colab.research.google.com/github/hn-iiitd/ML_Project/blob/madhav/ML_project_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
!wget https://raw.githubusercontent.com/hn-iiitd/ML_Project/refs/heads/main/online_gaming_behavior_dataset_updated.csv

--2024-11-28 18:41:37--  https://raw.githubusercontent.com/hn-iiitd/ML_Project/refs/heads/main/online_gaming_behavior_dataset_updated.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3134220 (3.0M) [text/plain]
Saving to: ‘online_gaming_behavior_dataset_updated.csv’

online_gaming_behav 100%[===================>]   2.99M  --.-KB/s    in 0.1s    

2024-11-28 18:41:38 (24.9 MB/s) - ‘online_gaming_behavior_dataset_updated.csv’ saved [3134220/3134220]



In [6]:
df = pd.read_csv("./online_gaming_behavior_dataset_updated.csv")
df.head()

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel,EstimatedAnnualIncome
0,9000,43,Male,Other,Strategy,16.271119,0,Medium,6,108,79,25,Medium,73408
1,9001,29,Female,USA,Strategy,5.525961,0,Medium,5,144,11,10,Medium,50308
2,9002,22,Female,USA,Sports,8.223755,0,Easy,16,142,35,41,High,83895
3,9003,35,Male,USA,Action,5.265351,1,Easy,9,85,57,47,Medium,50722
4,9004,33,Male,Europe,Action,15.531945,0,Medium,2,131,95,37,Medium,63269


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


label_encoder = LabelEncoder()
ToEncode = ['Gender','Location','GameGenre','GameDifficulty','EngagementLevel']
for col in ToEncode:
    df[col] = label_encoder.fit_transform(df[col])

X = df.drop(columns=['PlayerID', 'InGamePurchases'])
y = df['InGamePurchases']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
x_smote, y_smote = smote.fit_resample(X_train, y_train)
tomek = TomekLinks()
x_tomek, y_tomek = tomek.fit_resample(X_train, y_train)



In [9]:
type(x_smote)

pandas.core.frame.DataFrame

In [11]:
from sklearn.utils import shuffle

# Combine the feature sets and labels
x_combined = pd.concat([pd.DataFrame(x_smote), pd.DataFrame(x_tomek)], ignore_index=True)
y_combined = pd.concat([pd.Series(y_smote), pd.Series(y_tomek)], ignore_index=True)

# Shuffle the combined dataset
x_combined, y_combined = shuffle(x_combined, y_combined, random_state=42)

In [13]:
x_new,y_new = shuffle(x_combined,y_combined,random_state=270)

In [17]:
# Select the top 25,000 entries
top_25000_x = x_new.iloc[:25000]
top_25000_y = y_new.iloc[:25000]

# Combine the features and labels into a single DataFrame
top_25000 = pd.concat([top_25000_x, top_25000_y.rename("InGamePurchases")], axis=1)

# Save to CSV
top_25000.to_csv("random_probab_sampling.csv", index=False)


In [20]:
# Determine step size for 25,000 entries
total_data_points = len(x_combined)
desired_samples = 25000

# Calculate step size
step_size = total_data_points // desired_samples

# Ensure exactly 25,000 entries by slicing up to the required count
x_sampled = x_combined.iloc[:desired_samples * step_size:step_size]
y_sampled = y_combined.iloc[:desired_samples * step_size:step_size]

# If sampling results in more than 25,000 entries, truncate
x_sampled = x_sampled.iloc[:desired_samples]
y_sampled = y_sampled.iloc[:desired_samples]

# Combine features and target
sampled_data = pd.concat([x_sampled.reset_index(drop=True),
                          y_sampled.reset_index(drop=True).rename("InGamePurchases")], axis=1)

# Save to CSV
sampled_data.to_csv("systematic_sampled_entries.csv", index=False)



In [21]:
print(x_sampled.shape)

(25000, 12)
